In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import pickle
import random

from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation
from keras.optimizers import SGD

In [8]:
to_load=open("intents.json").read()
intents=json.loads(to_load)

In [9]:
words=[]
doc=[]
tags=[]
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w=word_tokenize(pattern)
        words.extend(w)
        doc.append((w,intent['tag']))
        if intent['tag'] not in tags:
            tags.append(intent['tag'])

In [10]:
l=WordNetLemmatizer()
ignore=['?','!','.']
words=[l.lemmatize(w.lower()) for w in words if w not in ignore]

In [11]:
words=sorted(list(set(words)))
tags=sorted(list(set(tags)))

In [12]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(tags,open('tags.pkl','wb'))

### CREATE TRAIN DATA

In [13]:
train=[]
output_zeros=[0]*len(tags)
for d in doc:
    bag=[]
    pattern_words=d[0]
    pattern_words=[l.lemmatize(w.lower()) for w in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output=list(output_zeros)
    output[tags.index(d[1])]=1
    train.append([bag,output])

In [14]:
random.shuffle(train)
train=np.array(train)
train_x=list(train[:,0])
train_y=list(train[:,1])

### Create a model.

#### This model has 3 layers. 1st layer has 128 neurons. 2nd layer has 64 neurons. 3rd one is the number of tags

In [15]:
model=Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation='softmax'))

In [16]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [17]:
%%time
hist=model.fit(np.array(train_x),np.array(train_y),epochs=200,batch_size=5,verbose=1)
model.save('chatbot_model.h5',hist)


Epoch 1/200
97/97 [==============================] - 0s 2ms/step - loss: 2.5342 - accuracy: 0.1031
Epoch 2/200
97/97 [==============================] - 0s 412us/step - loss: 2.3489 - accuracy: 0.2371
Epoch 3/200
97/97 [==============================] - 0s 412us/step - loss: 2.1339 - accuracy: 0.3299
Epoch 4/200
97/97 [==============================] - 0s 412us/step - loss: 1.8723 - accuracy: 0.3918
Epoch 5/200
97/97 [==============================] - 0s 330us/step - loss: 1.7819 - accuracy: 0.4433
Epoch 6/200
97/97 [==============================] - 0s 358us/step - loss: 1.5795 - accuracy: 0.4948
Epoch 7/200
97/97 [==============================] - 0s 330us/step - loss: 1.2688 - accuracy: 0.5773
Epoch 8/200
97/97 [==============================] - 0s 412us/step - loss: 1.2430 - accuracy: 0.5567
Epoch 9/200
97/97 [==============================] - 0s 412us/step - loss: 1.0681 - accuracy: 0.7010
Epoch 10/200
97/97 [==============================] - 0s 412us/step - loss: 0.9952 - accurac

97/97 [==============================] - 0s 412us/step - loss: 0.0683 - accuracy: 0.9794
Epoch 80/200
97/97 [==============================] - 0s 330us/step - loss: 0.0788 - accuracy: 0.9794
Epoch 81/200
97/97 [==============================] - 0s 330us/step - loss: 0.0568 - accuracy: 0.9691
Epoch 82/200
97/97 [==============================] - 0s 412us/step - loss: 0.0603 - accuracy: 0.9897
Epoch 83/200
97/97 [==============================] - 0s 330us/step - loss: 0.0866 - accuracy: 0.9794
Epoch 84/200
97/97 [==============================] - 0s 412us/step - loss: 0.0357 - accuracy: 1.0000
Epoch 85/200
97/97 [==============================] - 0s 412us/step - loss: 0.0526 - accuracy: 0.9794
Epoch 86/200
97/97 [==============================] - 0s 412us/step - loss: 0.0618 - accuracy: 0.9794
Epoch 87/200
97/97 [==============================] - 0s 412us/step - loss: 0.0613 - accuracy: 0.9897
Epoch 88/200
97/97 [==============================] - 0s 330us/step - loss: 0.0262 - accuracy: 

97/97 [==============================] - 0s 330us/step - loss: 0.0363 - accuracy: 1.0000
Epoch 159/200
97/97 [==============================] - 0s 330us/step - loss: 0.0076 - accuracy: 1.0000
Epoch 160/200
97/97 [==============================] - 0s 330us/step - loss: 0.0180 - accuracy: 0.9897
Epoch 161/200
97/97 [==============================] - 0s 330us/step - loss: 0.0298 - accuracy: 0.9897
Epoch 162/200
97/97 [==============================] - 0s 330us/step - loss: 0.0106 - accuracy: 1.0000
Epoch 163/200
97/97 [==============================] - 0s 412us/step - loss: 0.0519 - accuracy: 0.9897
Epoch 164/200
97/97 [==============================] - 0s 330us/step - loss: 0.0107 - accuracy: 1.0000
Epoch 165/200
97/97 [==============================] - 0s 412us/step - loss: 0.1247 - accuracy: 0.9794
Epoch 166/200
97/97 [==============================] - 0s 330us/step - loss: 0.0117 - accuracy: 1.0000
Epoch 167/200
97/97 [==============================] - 0s 330us/step - loss: 0.0198 - a